In [ ]:
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
import os

profile_name = os.environ['AWS_PROFILE_NAME']
region_name = os.environ['AWS_REGION_NAME']
bucket_prefix = os.environ['AWS_BUCKET_PREFIX']
role = os.environ['AWS_ROLE']

boto_sess = boto3.Session(profile_name=profile_name, region_name=region_name)
local_session = LocalSession(boto_sess)

bucket = bucket_prefix
role = role

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig, DebuggerHookConfig, CollectionConfig

## Not working tensorboard near real time debugging in local mode.
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path='s3://{}/tensorboard'.format(bucket),
    container_local_output_path='/opt/ml/output/tensorboard'
)

pytorch_estimator = PyTorch('train.py',
                            source_dir='./src',
                            sagemaker_session=local_session,
                            role=role,
                            instance_type='local',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            tensorboard_output_config=tensorboard_output_config,
                            hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})

pytorch_estimator.fit({'train': 's3://{}/dataset2'.format(bucket),
                       'test': 's3://{}/dataset2'.format(bucket)})

In [ ]:
tensorboard_output_path = pytorch_estimator.latest_job_tensorboard_artifacts_path()
print(tensorboard_output_path)



